In [38]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import itertools

In [ ]:
df = pd.read_csv(r"C:\Users\user\Desktop\hw_aa.csv", sep = ';') #Читаем датафрэйм 
df.head()

,Unnamed: 0,uid,experimentVariant,version,purchase
0,1,c4ca4238a0b923820dcc509a6f75849b,1,v2.8.0,0
1,2,c81e728d9d4c2f636f067f89cc14862c,0,v2.9.0,0
2,3,eccbc87e4b5ce2fe28308fd9f2a7baf3,1,v2.9.0,0
3,4,a87ff679a2f3e71d9181a67b7542122c,1,v2.8.0,0
4,5,e4da3b7fbbce2345d7772b0674a318d5,1,v2.8.0,0


In [ ]:
def metric_fpr(df_x, df_y, metric_col, n_sim, n_s_perc, n_s_min, estimator, *args, **kwargs): #Пишем фкнкцию которая будет считать показатель ложноположительных прогнозов
    stat_res = {
        'aa': {
            'pvalue': [],
            'mu_x':[],
            'mu_y':[]},
        'fpr':{
            'fpr_95': 0
        } 
    }#Создаём словарь, в котором будем хранить рассчитанные показатели
    for sin in range(n_sim): #Запускаем цикл симуляций А/А теста
        x = df_x[metric_col].sample(int(min(n_s_min, len(df_x) * n_s_perc)), replace = False).values
        y = df_y[metric_col].sample(int(min(n_s_min, len(df_y) * n_s_perc)), replace = False).values #Записываем в отдельные переменные результаты симуляций
        if estimator == 'prop': 
            counts = np.array([sum(x), sum(y)])
            nobs = np.array([len(x), len(y)])
            stat, pvalue = proportions_ztest(counts, nobs, *args, **kwargs) #Для оценщика proreties записываем расчёт pvalue при помощи proportions_ztest
        if estimator == 'ttest':
            stat, pvalue = stats.ttest_ind(x, y, *args, **kwargs) #Для оценщика ttest записываем расчёт pvalue при помощи ttest_ind
        stat_res['aa']['pvalue'].append(pvalue)
        stat_res['aa']['mu_x'].append(np.mean(x))
        stat_res['aa']['mu_y'].append(np.mean(y)) #Добавляем в словарь рассчитанные pvalue и средние
    stat_res['fpr']['fpr_95'] = float(sum(np.array(stat_res['aa']['pvalue']) <= 0.05) / n_sim) #Рассчитываем fpr
    return stat_res



In [ ]:
def fpr_report (df, metric_col, variation_col, group_col, n_sim, n_s_perc, n_s_min, estimator, *args, **kwargs): #Создаём функцию, которая рассчитает fpr для каждой версии продукта
    list_fpr = [] #Создаём список для записи расчётных значений 
    list_group = list(pd.unique(df[group_col])) #Создаём список уникальных версий продукта
    for v in range(len(list_group)): #Создаём цикл, рассчитывающий необходимые метрики для каждой версии продукта
        df_x = df[(df[variation_col] == 0) & (df[group_col] == list_group[v])] 
        df_y = df[(df[variation_col] == 1) & (df[group_col] == list_group[v])] #Создаём выборки, интересующей нас метрики и необходимой версией
        cr_x = sum(df_x[metric]) / len(df_x)
        cr_y = sum(df_y[metric]) / len(df_y) #Рассчитываем конверсию
        fpr = {}
        fpr = metric_fpr(
            df_x = df_x,
            df_y = df_y,
            metric_col = metric_col,
            n_sim = n_sim,
            n_s_perc = n_s_perc,
            n_s_min = n_s_min,
            estimator = estimator, *args, **kwargs
        ) #Запускаем ранее написанную функцию
        is_fpr = (fpr['fpr']['fpr_95']<= 0.05) #Проверяем результаты на статистическую значимость
        list_fpr.append([list_group[v], cr_x, cr_y, fpr['fpr']['fpr_95'], is_fpr]) #Добавляем полученные метрики в список
    report = pd.DataFrame.from_records(list_fpr, columns = ['group', 'cr_x', 'cr_y', 'fpr_95', 'is_fpr']) #Преобразуем список в датафрэйм
    return print(report)


In [ ]:
n_sim = 1000
n_s_perc = 0.9
n_s_min = 1000
metric = 'purchase'
vaiant = 'experimentVariant'
group = 'version'
res = fpr_report(
    df = df, 
    metric_col = metric,
    variation_col = vaiant,
    group_col = group, 
    n_sim = n_sim, 
    n_s_perc = n_s_perc, 
    n_s_min = n_s_min, 
    estimator = 'ttest'
) #Запускаем написанную функцию
#Результат: статистически значимо отклоняется версия приложения 2.8.0 и 3.8.0.0, но последняя отклоняется незначительно

      group      cr_x      cr_y  fpr_95  is_fpr
0    v2.8.0  0.000993  0.045606   1.000   False
1    v2.9.0  0.074658  0.071304   0.041    True
2  v3.7.4.0  0.059943  0.063018   0.033    True
3  v3.8.0.0  0.057604  0.062848   0.058   False
